matrix transformation only for perfect file (with path 0)

In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.express as px
from scipy.optimize import minimize

# pymatgen libraries
from pymatgen.core.structure import Structure

# import function from .py file
from get_weirdos_ver3_new_cleaned import get_dx_dz_init, get_sum_weirdos_Li_var
# from get_weirdos import get_sum_weirdos_Li
# from modify_dx_dz import create_file_name, change_dx_dz
# from get_weirdos import get_dx1_dx2_dz_init, get_sum_weirdos_Li

In [ ]:
# from get_weirdos_ver3_new_cleaned import varying_radius_vs_sumweirdosLi, plot_varying_radius_vs_sumweirdosLi

In [ ]:
direc = os.getcwd() # get current working directory

# # filefile_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_type2.cif"_path_ori_ref_48n24 = "./perfect_poscar/cif_matrix/ori/Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"  # Replace with your file path
  # Replace with your file path
# file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type5_ver1_a"
file_perfect_poscar_48n24_wo_cif_array = ["Li6PS5Cl_optitype1n2_strictr_a_type4_weirdo1_var2_opti_zoomedin"]
# file_perfect_poscar_48n24_wo_cif_array = ["Li6PS5Cl_type5_ver1_a_less_opti", "Li6PS5Cl_type5_ver1_b_less_opti", "Li6PS5Cl_type5_ver2_a_less_opti", "Li6PS5Cl_type5_ver2_b_less_opti", 
#                                           "Li6PS5Cl_type5_ver3_a_less_opti", "Li6PS5Cl_type5_ver3_b_less_opti", "Li6PS5Cl_type5_ver4_a_less_opti", "Li6PS5Cl_type5_ver4_b_less_opti", 
#                                           "Li6PS5Cl_type5_ver1_typ1_less_opti", "Li6PS5Cl_type5_ver1_typ2_less_opti", "Li6PS5Cl_type5_ver2_typ1_less_opti", "Li6PS5Cl_type5_ver2_typ2_less_opti",
#                                           "Li6PS5Cl_type5_ver3_typ1_less_opti", "Li6PS5Cl_type5_ver3_typ2_less_opti", "Li6PS5Cl_type5_ver4_typ1_less_opti", "Li6PS5Cl_type5_ver4_typ2_less_opti"]
file_perfect_poscar_24_wo_cif = "Li6PS5Cl_24_mod_2p27291"

# "Li6PS5Cl_type5_ver1_a", "Li6PS5Cl_type5_ver1_b", "Li6PS5Cl_type5_ver2_a", "Li6PS5Cl_type5_ver2_b", 
#                                           "Li6PS5Cl_type5_ver3_a", "Li6PS5Cl_type5_ver3_b", "Li6PS5Cl_type5_ver4_a", "Li6PS5Cl_type5_ver4_b", 
#                                           "Li6PS5Cl_type5_ver1_typ1", "Li6PS5Cl_type5_ver1_typ2", "Li6PS5Cl_type5_ver2_typ1", "Li6PS5Cl_type5_ver2_typ2",
#                                           "Li6PS5Cl_type5_ver3_typ1", "Li6PS5Cl_type5_ver3_typ2", "Li6PS5Cl_type5_ver4_typ1", "Li6PS5Cl_type5_ver4_typ2",
                                          # "Li6PS5Cl_type5_ver1_a_less", "Li6PS5Cl_type5_ver1_b_less", "Li6PS5Cl_type5_ver2_a_less", "Li6PS5Cl_type5_ver2_b_less", 
                                          # "Li6PS5Cl_type5_ver3_a_less", "Li6PS5Cl_type5_ver3_b_less", "Li6PS5Cl_type5_ver4_a_less", "Li6PS5Cl_type5_ver4_b_less", 
                                          # "Li6PS5Cl_type5_ver1_typ1_less", "Li6PS5Cl_type5_ver1_typ2_less", "Li6PS5Cl_type5_ver2_typ1_less", "Li6PS5Cl_type5_ver2_typ2_less",
                                          # "Li6PS5Cl_type5_ver3_typ1_less", "Li6PS5Cl_type5_ver3_typ2_less", "Li6PS5Cl_type5_ver4_typ1_less", "Li6PS5Cl_type5_ver4_typ2_less"]

# file_ori_ref_48n24 = "Li6PS5Cl_48n24_mod_old_2p27291_0p88814_arcA.cif"

results_folder = "_results"
iter_type = "varying_radius"

litype = 4
activate_radius = 1
max_mapping_radius = 0.075 # 0.025 # 0.043 # 0.035
max_mapping_radius_48htype2 = 0.075 # 0.075 # 0.076 # 0.071 

var_excel_file = "wo_outliars"
max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

folder_name_iter_type = f"/{results_folder}/_{iter_type}/"
path_folder_name_iter_type = direc+str(folder_name_iter_type)

In [ ]:
delta_radius = 0.001 
n_sample = 114
var_optitype = "varying_radius"

# radius_sumweirdosLi_df= varying_radius_vs_sumweirdosLi(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, delta_radius, n_sample, var_optitype)
# plot_varying_radius_vs_sumweirdosLi(radius_sumweirdosLi_df)

In [ ]:
for file_perfect_poscar_48n24_wo_cif in file_perfect_poscar_48n24_wo_cif_array:
    file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

    col_radius_type1 = "radius_type1"
    col_radius_type2 = "radius_type2"
    col_sumweirdosLi = "sumweirdosLi"

    radius_sumweirdosLi_df = pd.DataFrame()
    radius_sumweirdosLi_df[col_radius_type1] = None
    radius_sumweirdosLi_df[col_radius_type2] = None
    radius_sumweirdosLi_df[col_sumweirdosLi] = None

    idx_sumweirdosLi_df = 0
    # forward_max_mapping_radius_init = max_mapping_radius
    # forward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2
    backward_max_mapping_radius_init = max_mapping_radius
    backward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

    # for i in range(int(n_sample/2)):
    #     forward_max_mapping_radius = forward_max_mapping_radius_init + (i * delta_radius)
    #     # forward_max_mapping_radius_48htype2 = forward_max_mapping_radius_init + (i * delta_radius)
    #     forward_max_mapping_radius_48htype2 = forward_max_mapping_radius_48htype2_init

    #     # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, forward_max_mapping_radius, forward_max_mapping_radius_48htype2, var_optitype)
    #     sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, forward_max_mapping_radius, forward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)

    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type1] = forward_max_mapping_radius
    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type2] = forward_max_mapping_radius_48htype2
    #     radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li
        
    #     idx_sumweirdosLi_df += 1
    #     print(idx_sumweirdosLi_df)

    for j in range(int(n_sample/2)):
        backward_max_mapping_radius = backward_max_mapping_radius_init - (j * delta_radius)
        backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init - (j * delta_radius)
        # backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init
        # backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init

        # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, backward_max_mapping_radius, backward_max_mapping_radius_48htype2, var_optitype)
        # sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, backward_max_mapping_radius, backward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_perfect_poscar_48n24, litype, var_optitype)
        sum_weirdos_Li = get_sum_weirdos_Li_var(backward_max_mapping_radius, backward_max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24_wo_cif, file_perfect_poscar_48n24_wo_cif, litype, var_optitype, iter_type = "varying_radius", foldermapping_namestyle_all = True, cif_namestyle_all = None, modif_all_litype = None, full_calculation=True)
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type1] = backward_max_mapping_radius
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_radius_type2] = backward_max_mapping_radius_48htype2
        radius_sumweirdosLi_df.at[idx_sumweirdosLi_df, col_sumweirdosLi] = sum_weirdos_Li

        idx_sumweirdosLi_df += 1
        print(idx_sumweirdosLi_df)

    min_sumweirdos = int(radius_sumweirdosLi_df["sumweirdosLi"].min())
    radius_sumweirdosLi_df.to_excel(f'{path_folder_name_iter_type}07_varying_radius_{min_sumweirdos}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)


In [ ]:
el = "Li"

for file_perfect_poscar_48n24_wo_cif in file_perfect_poscar_48n24_wo_cif_array:
    file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

    col_radius_type1 = "radius_type1"
    col_radius_type2 = "radius_type2"
    col_amount_empty = "amount_empty"

    radius_amount_empty_df = pd.DataFrame()
    radius_amount_empty_df[col_radius_type1] = [0 for _ in range(int(n_sample/2))]
    radius_amount_empty_df[col_radius_type2] = [0 for _ in range(int(n_sample/2))]
    radius_amount_empty_df[col_amount_empty] = [0 for _ in range(int(n_sample/2))]

    idx_amount_empty_df = 0
    # forward_max_mapping_radius_init = max_mapping_radius
    # forward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2
    backward_max_mapping_radius_init = max_mapping_radius
    backward_max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

    for j in range(int(n_sample/2)):
        df = pd.DataFrame()

        backward_max_mapping_radius = backward_max_mapping_radius_init - (j * delta_radius)
        backward_max_mapping_radius_48htype2 = backward_max_mapping_radius_48htype2_init - (j * delta_radius)

        if activate_radius == 1:
            file_loc_important_cols = pd.read_pickle(f'{path_folder_name_iter_type}{file_perfect_poscar_48n24_wo_cif}/file_loc_important_cols_{backward_max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.pkl') 
        elif activate_radius == 2:
            file_loc_important_cols = pd.read_pickle(f'{path_folder_name_iter_type}{file_perfect_poscar_48n24_wo_cif}/file_loc_important_cols_{backward_max_mapping_radius}_{backward_max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')
        # elif activate_radius == 3:
        #     file_loc_important_cols = pd.read_pickle(f'{path_folder_name_iter_type}file_loc_important_cols_{max_mapping_radius}_{max_mapping_radius_48htype2}_{max_mapping_radius_48htype1_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')

        col_top1_sorted_idxweirdo_label_el = f"top1_sorted_idxweirdo_label_{el}"

        data = file_loc_important_cols[col_top1_sorted_idxweirdo_label_el]

        df = pd.DataFrame(data.items(), columns=['index', 'idxweirdo_label'])

        # Count empty dictionaries
        amount_empty = df['idxweirdo_label'].apply(lambda x: len(x) == 0).sum()

        radius_amount_empty_df.at[j, col_radius_type1] = backward_max_mapping_radius
        radius_amount_empty_df.at[j, col_radius_type2] = backward_max_mapping_radius_48htype2
        radius_amount_empty_df.at[j, col_amount_empty] = amount_empty

In [ ]:
radius_amount_empty_df

In [ ]:
fig = px.bar(radius_amount_empty_df, x="radius_type1", y="amount_empty", title="Amount of folder w/o weirdo")
fig.show()

In [ ]:
radius_sumweirdosLi_df

In [ ]:
radius_sumweirdosLi_df.plot(x="radius_type1", y="sumweirdosLi", title = "Radius vs amount of weirdos")

In [ ]:
# min_sumweirdos = int(radius_sumweirdosLi_df["sumweirdosLi"].min())

In [ ]:
# path_folder_name_iter_type

In [ ]:
# radius_sumweirdosLi_df.to_excel(f'{path_folder_name_iter_type}07_varying_radius_{min_sumweirdos}_{file_perfect_poscar_48n24_wo_cif}_{var_excel_file}.xlsx', index=False)

In [ ]:
# radius_sumweirdosLi_df(radius_sumweirdosLi_df)